In [48]:
import pandas as pd
import cohere
df = pd.read_csv('/Users/saikrishna/Documents/Work/datab.csv')

In [49]:
print(df)

            Username             CreatedAt  \
0            savolai  2011-06-11T18:04:13Z   
1           jantatje  2014-04-21T09:39:45Z   
2           dharnnie  2016-11-29T10:15:26Z   
3             zengsn  2011-02-20T05:48:40Z   
4      shriroopjoshi  2013-08-11T11:06:26Z   
5            benedly  2016-06-11T03:55:27Z   
6         ovidiubute  2010-11-01T10:54:43Z   
7        cihanzengin  2012-02-05T12:02:34Z   
8    Joshua-Anderson  2013-02-10T18:25:37Z   
9             Physwf  2012-12-09T10:07:26Z   
10       kyanagimoto  2010-09-05T10:02:31Z   
11          5kylight  2014-12-27T16:24:35Z   
12          hachreak  2010-08-26T15:49:02Z   
13  MarcinFrankowski  2014-04-30T10:06:25Z   
14             srand  2011-11-09T17:58:21Z   
15         jlhuacles  2014-09-23T05:39:35Z   
16         SpiraMira  2010-05-24T23:50:47Z   
17      ryancatalani  2010-08-16T02:40:44Z   
18            asgrim  2010-11-25T08:54:08Z   
19       dominictarr  2010-04-29T05:42:50Z   
20            drewag  2011-06-12T2

In [50]:
language_columns = [col for col in df.columns if '%' in col]
df['Skills'] = df.apply(lambda row: {lang.replace('%', ''): row[lang] for lang in language_columns if row[lang] > 0}, axis=1)

In [51]:
def generate_summary(row):
    skills = row['Skills']
    sorted_skills = sorted(skills.items(), key=lambda x: x[1], reverse=True)
    skill_descriptions = [f"{skill} ({percentage}%)" for skill, percentage in sorted_skills]
    return f"User {row['Username']} is proficient in {', '.join(skill_descriptions)}."
df['SkillSummary'] = df.apply(generate_summary, axis=1)


In [52]:
print(df)

            Username             CreatedAt  \
0            savolai  2011-06-11T18:04:13Z   
1           jantatje  2014-04-21T09:39:45Z   
2           dharnnie  2016-11-29T10:15:26Z   
3             zengsn  2011-02-20T05:48:40Z   
4      shriroopjoshi  2013-08-11T11:06:26Z   
5            benedly  2016-06-11T03:55:27Z   
6         ovidiubute  2010-11-01T10:54:43Z   
7        cihanzengin  2012-02-05T12:02:34Z   
8    Joshua-Anderson  2013-02-10T18:25:37Z   
9             Physwf  2012-12-09T10:07:26Z   
10       kyanagimoto  2010-09-05T10:02:31Z   
11          5kylight  2014-12-27T16:24:35Z   
12          hachreak  2010-08-26T15:49:02Z   
13  MarcinFrankowski  2014-04-30T10:06:25Z   
14             srand  2011-11-09T17:58:21Z   
15         jlhuacles  2014-09-23T05:39:35Z   
16         SpiraMira  2010-05-24T23:50:47Z   
17      ryancatalani  2010-08-16T02:40:44Z   
18            asgrim  2010-11-25T08:54:08Z   
19       dominictarr  2010-04-29T05:42:50Z   
20            drewag  2011-06-12T2

In [55]:
import cohere
co = cohere.Client('ycZhdKsC3Vy5BRzRZuIXvZi7F27bPTWBrZKDzik6')


In [56]:
def identify_weakness(summary):
    prompt = f"{summary}\n\nIdentify the user's weaknesses and areas for improvement."
    response = co.generate(model='command', prompt=prompt, max_tokens=50)
    return response.generations[0].text.strip()
df['Weaknesses'] = df['SkillSummary'].apply(identify_weakness)

In [57]:
embeddings = co.embed(texts=df['SkillSummary'].tolist(), model='large').embeddings
df['Embeddings'] = embeddings

In [58]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

similarity_matrix = cosine_similarity(embeddings)


In [59]:
def find_complementary_users(user_index):
    user_similarities = similarity_matrix[user_index]
    complementary_indices = np.argsort(user_similarities)
    # Exclude the user themselves
    complementary_indices = complementary_indices[complementary_indices != user_index]
    return df.iloc[complementary_indices[:5]]['Username'].tolist()
df['ComplementaryUsers'] = df.index.map(find_complementary_users)

In [60]:
def generate_recommendation(row):
    recommendation = f"User {row['Username']} could improve in {row['Weaknesses']}.\n"
    recommendation += f"Potential teammates to complement their skills: {', '.join(row['ComplementaryUsers'])}."
    return recommendation
df['Recommendation'] = df.apply(generate_recommendation, axis=1)


In [61]:
df[['Username', 'Recommendation']].to_csv('recommendations1.csv', index=False)
